In [1]:
import sys
sys.path.append('../../')
sys.path.append('../')

import fm
from torch.profiler import profile, record_function, ProfilerActivity


In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
import torch.fx as fx
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [3]:
import time
import torch._dynamo as dynamo


In [4]:
def gen_pattern_replace_and_matcher_for_FM(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

  class PatternClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = torch.nn.Embedding(1, 1)

    def forward(self,x):
      x = self.embed(x)
      square_of_sum = torch.sum(x, dim=1) ** 2
      sum_of_square = torch.sum(x ** 2, dim=1)
      ix = square_of_sum - sum_of_square    
      ix = torch.sum(ix, dim=1, keepdim=True)  
      return 0.5 * ix
  def _match(match,ori,pat):
    return True 
  # env  = utils.get_env(traced)
  # target_node = env[target_node_name]
  # target_node_mod = utils.get_target_mod(traced,target_node_name,"_")
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  

  
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      embedding_config = embed_node_module.weight.data.shape
      self.embed = nn.Embedding(embedding_config[0],embedding_config[1])
      self.embed.weight.data.copy_(embed_node_module.weight.data)
    def forward(self,x):
      redency_part = x[redency_part_slice] 
      unredency_part = x[unredency_part_slice] 
      redency_embed = self.embed(redency_part)
      unredency_embed = self.embed(unredency_part)
      redency_embed_sum = torch.sum(redency_embed,dim=0)
      unredency_embed_sum = torch.sum(unredency_embed,dim=1)
      square_of_sum = (redency_embed_sum + unredency_embed_sum) ** 2
      redency_embed_square_sum = torch.sum(redency_embed ** 2,dim=0)
      unredency_embed_square_sum = torch.sum(unredency_embed ** 2,dim=1)
      sum_of_square = redency_embed_square_sum + unredency_embed_square_sum
      ix = square_of_sum - sum_of_square
      ix = torch.sum(ix,dim = 1,keepdim=True)
      return 0.5 * ix
    
  
  return pattern,ReplacementClass(),_match

In [5]:
def workload_FM(num_field, prefix,dim = 64):
  # print(f"now gen workload of wdl with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}")
  model_ori = fm.FactorizationMachineModel([100 for i in range(num_field)],dim,)
  ori_traced = symbolic_trace(model_ori)
  
  model_modify = fm.FactorizationMachineModel([100 for i in range(num_field)],dim,)
  modify_traced = symbolic_trace(model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_FM(modify_traced,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      )
  matches = subgraph_rewriter.replace_pattern_with_filters(modify_traced, pattern, replace,[match])
  return ori_traced,modify_traced

In [8]:
def genWorkload(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64):
  ori_model_name = f'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/fm_rate/fm_{batch}_{num_field}_{prefix}_{dim}_ori.onnx'
  modify_model_name = f'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/fm_rate/fm_{batch}_{num_field}_{prefix}_{dim}_modify.onnx'
  ori, modify = workload_FM(num_field = num_field,prefix = prefix,  dim = dim)
  torch.onnx.export(ori,               # 模型 being run
                  torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  ori_model_name,        # 导出文件的文件名
                  export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
                  opset_version=10,   # ONNX版本
                  do_constant_folding=True,  # 是否执行常量折叠以优化模型
                  input_names = ['input'],   # 输入的名称
                  output_names = ['output'], # 输出的名称
                  )
  torch.onnx.export(modify,               # 模型 being run
                  torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  modify_model_name,        # 导出文件的文件名
                  export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
                  opset_version=10,   # ONNX版本
                  do_constant_folding=True,  # 是否执行常量折叠以优化模型
                  input_names = ['input'],   # 输入的名称
                  output_names = ['output'], # 输出的名称
                  )

In [ ]:
dims= [32]
batches = [1024,2048,4096]
num_field_and_prefixs = [(34 * 5,29*5),(22 * 5,10 * 5),(34 ,29),(22 ,10 )]

In [9]:
for dim in dims:
  for batch in batches:
    for num_field,prefix in num_field_and_prefixs:
      genWorkload(num_field=num_field,prefix=prefix,batch=batch,dim=dim)

NameError: name 'num_field_and_prefixs' is not defined

In [10]:
dims= [32]
batches = [1024,2048,4096]
for dim in dims:
  for batch in batches:
    for prefix in [10,20,30,40,50,60,70,80,90,99]:
      genWorkload(num_field=100,prefix=prefix,batch=batch,dim=dim)

/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: Us

In [18]:
def calculate_mean_and_variance_manual(data):
    n = len(data)
    mean = sum(data) / n
    variance = sum((x - mean) ** 2 for x in data) / n
    return mean, variance

In [19]:
def gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32, workload_func = workload_FM):
  def run(model):
    traced_model = torch.jit.trace(model, torch.randint(low=0, high=88, size=(batch,num_field), dtype=torch.long))
    compiled_model = torch.compile(traced_model, backend="inductor")
    compiled_model.eval()
    total_time = 0
    total_time = []
    t = torch.randint(low=0, high=88, size=(batch ,num_field), dtype=torch.long)

    with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
        with record_function("model_inference"):
          for i in range(100):
              start_time = time.time()  # 开始计时
              with torch.no_grad():
                soutput = compiled_model(t)
              end_time = time.time()  # 结束计时
              
              # 计算并打印函数执行所需的时间
              elapsed_time = end_time - start_time
              total_time.append(elapsed_time * 1000)
    print(calculate_mean_and_variance_manual(total_time))
  print(f"now gen workload of fm with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}, batch :{batch}")
  ori, modify = workload_func(num_field,prefix,dim)
  run(ori)
  run(modify)

In [131]:
gen_and_test(num_field = 22,prefix = 10, batch = 1024, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 22, prefix: 10, batch :1024


STAGE:2025-02-08 00:59:37 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:59:37 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:59:37 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.8416485786437988, 0.16393162656527238)


STAGE:2025-02-08 00:59:37 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:59:38 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:59:38 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.9055399894714355, 0.12827783950228877)


In [133]:
gen_and_test(num_field = 22,prefix = 10, batch = 2048, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 22, prefix: 10, batch :2048


STAGE:2025-02-08 00:59:42 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:59:43 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:59:43 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.0996079444885254, 0.10654750905700894)


STAGE:2025-02-08 00:59:43 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:59:43 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:59:43 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.1716532707214355, 0.12533544219763826)


In [134]:
gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 22, prefix: 10, batch :4096


STAGE:2025-02-08 00:59:45 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:59:45 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:59:45 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.1705207824707031, 0.3099678889157076)


STAGE:2025-02-08 00:59:46 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:59:46 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection


(1.3215446472167969, 0.2406660722954257)


STAGE:2025-02-08 00:59:46 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [135]:
gen_and_test(num_field = 22,prefix = 10, batch = 8192, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 22, prefix: 10, batch :8192


STAGE:2025-02-08 00:59:48 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:59:48 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:59:48 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.3257694244384766, 0.24388953520428913)


STAGE:2025-02-08 00:59:49 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:59:49 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:59:49 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.3782858848571777, 0.22229009957186463)


In [136]:
gen_and_test(num_field = 22,prefix = 10, batch = 1024, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 22, prefix: 10, batch :1024


STAGE:2025-02-08 00:59:52 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:59:52 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:59:52 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.1367392539978027, 0.06965451240716902)


STAGE:2025-02-08 00:59:52 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:59:52 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:59:52 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.2312746047973633, 0.11455207097696984)


In [25]:
gen_and_test(num_field = 22,prefix = 10, batch = 2048, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 22, prefix: 10, batch :2048


STAGE:2025-02-07 19:53:03 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:03 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:03 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.9548068046569824, 0.15408704072683577)


STAGE:2025-02-07 19:53:04 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:04 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:04 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.0307502746582031, 0.18146815486943524)


In [26]:
gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 22, prefix: 10, batch :4096


STAGE:2025-02-07 19:53:04 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:04 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:04 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.567110061645508, 3.382787081272909)


STAGE:2025-02-07 19:53:05 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:05 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:05 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.4825963973999023, 0.15916405393454625)


In [27]:
gen_and_test(num_field = 22,prefix = 10, batch = 8192, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 22, prefix: 10, batch :8192


STAGE:2025-02-07 19:53:06 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:06 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:06 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
STAGE:2025-02-07 19:53:07 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(9.518556594848633, 19.34084221043122)


STAGE:2025-02-07 19:53:07 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:07 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(6.235754489898682, 9.663418555163616)


In [144]:
gen_and_test(num_field = 34,prefix = 29, batch = 1024, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 34, prefix: 29, batch :1024


STAGE:2025-02-08 01:01:00 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 01:01:00 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 01:01:00 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.0046100616455078, 0.09465318785260024)


STAGE:2025-02-08 01:01:01 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 01:01:01 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 01:01:01 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.0007023811340332, 0.12646167576235712)


In [146]:
gen_and_test(num_field = 34,prefix = 29, batch = 2048, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 34, prefix: 29, batch :2048


STAGE:2025-02-08 01:01:07 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 01:01:07 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 01:01:07 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.9464263916015625, 0.11204329098291055)


STAGE:2025-02-08 01:01:07 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 01:01:08 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 01:01:08 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.1504411697387695, 0.14993648562722228)


In [147]:
gen_and_test(num_field = 34,prefix = 29, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 34, prefix: 29, batch :4096


STAGE:2025-02-08 01:01:10 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 01:01:10 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 01:01:10 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.3439488410949707, 0.1347106061245995)


STAGE:2025-02-08 01:01:11 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 01:01:11 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection


(1.3246703147888184, 0.16576934883119065)


STAGE:2025-02-08 01:01:11 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [31]:
gen_and_test(num_field = 34,prefix = 29, batch = 8192, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 34, prefix: 29, batch :8192


STAGE:2025-02-07 19:53:11 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:12 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:12 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
STAGE:2025-02-07 19:53:12 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(1.523454189300537, 0.13468056187662114)


STAGE:2025-02-07 19:53:12 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:12 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.4367294311523438, 0.2579140228590404)


In [32]:
gen_and_test(num_field = 34,prefix = 29, batch = 1024, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 34, prefix: 29, batch :1024


STAGE:2025-02-07 19:53:13 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:13 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:13 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.1168384552001953, 0.263013534458878)


STAGE:2025-02-07 19:53:13 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:13 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:13 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(0.9875202178955078, 0.16032600901780825)


In [33]:
gen_and_test(num_field = 34,prefix = 29, batch = 2048, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 34, prefix: 29, batch :2048


STAGE:2025-02-07 19:53:14 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:14 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:14 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.1469244956970215, 0.19664496036853052)


STAGE:2025-02-07 19:53:14 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:14 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:14 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.1899399757385254, 0.10601153969105326)


In [34]:
gen_and_test(num_field = 34,prefix = 29, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 34, prefix: 29, batch :4096


STAGE:2025-02-07 19:53:15 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:15 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:15 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
STAGE:2025-02-07 19:53:15 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(1.608266830444336, 0.7529647238243342)


STAGE:2025-02-07 19:53:16 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:16 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.4737462997436523, 0.18840984037069575)


In [35]:
gen_and_test(num_field = 34,prefix = 29, batch = 8192, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 34, prefix: 29, batch :8192


STAGE:2025-02-07 19:53:16 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:18 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:18 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(14.982514381408691, 24.531377205880744)


STAGE:2025-02-07 19:53:18 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:18 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:18 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.4224481582641602, 0.44050454832813557)


In [36]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 110, prefix: 50, batch :1024


STAGE:2025-02-07 19:53:19 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:19 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:19 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.2922930717468262, 0.08359043005725653)


STAGE:2025-02-07 19:53:19 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:19 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:19 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.2433910369873047, 0.1659640437537746)


In [37]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 110, prefix: 50, batch :2048


STAGE:2025-02-07 19:53:20 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:20 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:20 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.357567310333252, 0.11862930406891792)


STAGE:2025-02-07 19:53:21 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:21 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:21 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.3428258895874023, 0.7335291963727286)


In [113]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 110, prefix: 50, batch :4096


STAGE:2025-02-07 20:26:24 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:26:25 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:26:25 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.898378372192383, 0.3319453627227631)


STAGE:2025-02-07 20:26:25 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:26:25 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:26:25 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.8217110633850098, 0.22489647414545288)


In [114]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 8192, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 110, prefix: 50, batch :8192


STAGE:2025-02-07 20:26:44 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:26:47 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:26:47 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(27.14857578277588, 32.741037217397206)


STAGE:2025-02-07 20:26:47 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:26:48 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:26:48 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(4.137053489685059, 0.6568070191633524)


In [40]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 110, prefix: 50, batch :1024


STAGE:2025-02-07 19:53:28 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:28 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:28 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.9640564918518066, 0.6878889546158007)


STAGE:2025-02-07 19:53:29 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:29 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:29 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.5652275085449219, 0.1994717580600991)


In [41]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 110, prefix: 50, batch :2048


STAGE:2025-02-07 19:53:29 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:30 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:30 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.6950740814208984, 0.5970603580408351)


STAGE:2025-02-07 19:53:30 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:30 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:30 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.5949296951293945, 0.24020775160806807)


In [117]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 110, prefix: 50, batch :4096


STAGE:2025-02-07 20:27:42 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:27:45 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:27:45 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(31.65731906890869, 0.8585032960354511)


STAGE:2025-02-07 20:27:45 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:27:46 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:27:46 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(11.692490577697754, 0.2845200276851756)


In [43]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 8192, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 110, prefix: 50, batch :8192


STAGE:2025-02-07 19:53:35 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:41 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:41 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(57.32489585876465, 2.0474609015309397)


STAGE:2025-02-07 19:53:42 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:45 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:45 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(33.90780448913574, 2.246284015564015)


In [44]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :1024


STAGE:2025-02-07 19:53:46 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:46 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:46 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.6725349426269531, 0.3848434088013164)


STAGE:2025-02-07 19:53:46 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:46 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:46 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.164708137512207, 0.1876634144309719)


In [45]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :2048


STAGE:2025-02-07 19:53:47 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:47 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:47 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
STAGE:2025-02-07 19:53:47 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(2.2417330741882324, 1.1623654989932675)


STAGE:2025-02-07 19:53:48 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:48 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.3333487510681152, 0.288530038068302)


In [46]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :4096


STAGE:2025-02-07 19:53:48 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:49 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:49 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(5.362715721130371, 2.884875063978143)


STAGE:2025-02-07 19:53:49 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:49 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:49 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.395406723022461, 0.7451441537341452)


In [47]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 8192, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :8192


STAGE:2025-02-07 19:53:50 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:55 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:55 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(44.30685997009277, 1.2634602743673895)


STAGE:2025-02-07 19:53:55 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:55 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:55 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.9654035568237305, 1.335709188765577)


In [48]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 170, prefix: 145, batch :1024


STAGE:2025-02-07 19:53:56 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:56 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:56 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.695791721343994, 1.3220789974809577)


STAGE:2025-02-07 19:53:56 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:57 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:57 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.282038688659668, 0.13799362093322998)


In [49]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 170, prefix: 145, batch :2048


STAGE:2025-02-07 19:53:57 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:53:58 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:58 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
STAGE:2025-02-07 19:53:58 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(5.091557502746582, 1.8570046147715402)


STAGE:2025-02-07 19:53:58 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:53:58 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.3472771644592285, 0.37800713118372187)


In [50]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 170, prefix: 145, batch :4096


STAGE:2025-02-07 19:53:59 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:54:03 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:54:03 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(43.39672327041626, 1.3906020176079892)


STAGE:2025-02-07 19:54:04 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:54:04 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:54:04 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.7615699768066406, 0.25738419744811836)


In [128]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 8192, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 170, prefix: 145, batch :8192


STAGE:2025-02-07 20:31:16 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:31:25 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:31:25 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(85.56152105331421, 2.656289586246885)


STAGE:2025-02-07 20:31:26 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:31:27 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:31:27 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(17.30111598968506, 1.179403911942245)


In [127]:
gen_and_test(num_field = 100,prefix = 10, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 10, batch :4096


STAGE:2025-02-07 20:30:52 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:30:54 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:30:54 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(15.58389663696289, 0.270720742037156)


STAGE:2025-02-07 20:30:54 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:30:54 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:30:54 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.4332261085510254, 0.2918034632500621)


In [126]:
gen_and_test(num_field = 100,prefix = 20, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 20, batch :4096


STAGE:2025-02-07 20:30:49 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:30:51 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:30:51 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(15.703754425048828, 0.7753242890430556)


STAGE:2025-02-07 20:30:52 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:30:52 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:30:52 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.1569204330444336, 0.27175533384706796)


In [125]:
gen_and_test(num_field = 100,prefix = 30, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 30, batch :4096


STAGE:2025-02-07 20:30:47 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:30:48 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:30:48 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(14.414472579956055, 3.3435604073929426)


STAGE:2025-02-07 20:30:48 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:30:48 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:30:48 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.9005012512207031, 0.7007505201727326)


In [129]:
gen_and_test(num_field = 100,prefix = 40, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 40, batch :4096


STAGE:2025-02-07 20:31:36 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:31:38 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:31:38 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(16.057605743408203, 0.8357440145573491)


STAGE:2025-02-07 20:31:38 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:31:38 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:31:38 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.8061017990112305, 0.4902163757606104)


In [123]:
gen_and_test(num_field = 100,prefix = 50, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 50, batch :4096


STAGE:2025-02-07 20:30:41 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:30:42 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:30:42 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(14.10285472869873, 6.702689909411674)


STAGE:2025-02-07 20:30:43 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:30:43 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:30:43 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.5732431411743164, 0.3137766898589689)


In [122]:
gen_and_test(num_field = 100,prefix = 60, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 60, batch :4096


STAGE:2025-02-07 20:30:39 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:30:40 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:30:40 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.5649261474609375, 0.1645695871047792)


STAGE:2025-02-07 20:30:40 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:30:40 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:30:40 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.4886021614074707, 0.36051086269139887)


In [121]:
gen_and_test(num_field = 100,prefix = 70, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 70, batch :4096


STAGE:2025-02-07 20:30:38 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:30:38 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:30:38 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.718806266784668, 0.7372549981710108)


STAGE:2025-02-07 20:30:39 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:30:39 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:30:39 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.4331436157226562, 0.14862782441014133)


In [120]:
gen_and_test(num_field = 100,prefix = 80, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 80, batch :4096


STAGE:2025-02-07 20:30:36 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:30:37 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:30:37 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.8404688835144043, 0.7762883652674191)


STAGE:2025-02-07 20:30:37 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:30:37 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:30:37 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.6406798362731934, 0.35401794036147294)


In [119]:
gen_and_test(num_field = 100,prefix = 90, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 90, batch :4096


STAGE:2025-02-07 20:30:35 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:30:35 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:30:35 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.5604963302612305, 0.18206015568011935)


STAGE:2025-02-07 20:30:36 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:30:36 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:30:36 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.3909673690795898, 0.15843232179122424)


In [130]:
gen_and_test(num_field = 100,prefix = 99, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 99, batch :4096


STAGE:2025-02-07 20:33:52 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:33:54 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:33:54 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(15.424516201019287, 0.6036945852599729)


STAGE:2025-02-07 20:33:54 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 20:33:54 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 20:33:54 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.2678003311157227, 0.14349893651797174)


In [148]:
def gen_and_test_KFLOPS(num_field = 22,prefix = 10, batch = 4096, dim = 32, workload_func = workload_FM):
  def run(model):
    traced_model = torch.jit.trace(model, torch.randint(low=0, high=88, size=(batch,num_field), dtype=torch.long))
    compiled_model = torch.compile(traced_model, backend="inductor")
    compiled_model.eval()
    total_time = 0
    total_time = []
    t = torch.randint(low=0, high=88, size=(batch ,num_field), dtype=torch.long)

    with profile(activities=[ProfilerActivity.CPU], record_shapes=True,    with_flops=True,  ) as prof:
        with record_function("model_inference"):
          for i in range(100):
              start_time = time.time()  # 开始计时
              with torch.no_grad():
                soutput = compiled_model(t)
              end_time = time.time()  # 结束计时
              
              # 计算并打印函数执行所需的时间
              elapsed_time = end_time - start_time
              total_time.append(elapsed_time * 1000)
    print(calculate_mean_and_variance_manual(total_time))
    return prof
  print(f"now gen workload of fm with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}, batch :{batch}")
  ori, modify = workload_func(num_field,prefix,dim)
  p1 = run(ori)
  p2 = run(modify)
  return p1, p2

In [52]:
def gen_and_test_gpu(num_field = 22,prefix = 10, batch = 4096, dim = 32, workload_func = workload_FM):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  def run(model):
    model.to(device)
    traced_model = torch.jit.trace(model, torch.randint(low=0, high=88, size=(batch,num_field), dtype=torch.long).to(device)).to(device)
    compiled_model = torch.compile(traced_model, backend="inductor").to(device)
    compiled_model.eval()
    total_time = 0
    t = torch.randint(low=0, high=88, size=(batch ,num_field), dtype=torch.long).to(device)

    with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
        with record_function("model_inference"):
          for i in range(100):
              start_time = time.time()  # 开始计时
              with torch.no_grad():
                soutput = compiled_model(t)
              end_time = time.time()  # 结束计时
              
              # 计算并打印函数执行所需的时间
              elapsed_time = end_time - start_time
              total_time += elapsed_time
    print(total_time * 1000)
    return prof
  print(f"now gen workload of fm with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}, batch :{batch}")
  ori, modify = workload_func(num_field,prefix,dim)
  p1 = run(ori)
  p2 = run(modify)
  return p1, p2

In [53]:
p1, p2 = gen_and_test_gpu(num_field = 34,prefix = 29, batch = 8192, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 34, prefix: 29, batch :8192


STAGE:2025-02-07 19:54:15 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:54:16 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:54:16 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


269.34075355529785


STAGE:2025-02-07 19:54:16 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:54:16 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:54:16 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


268.4900760650635


In [54]:
print(p1.key_averages().table(sort_by="cuda_time_total"))


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference         0.00%       0.000us         0.00%       0.000us       0.000us      66.500ms        50.69%      66.500ms      66.500ms             1  
                                        model_inference        79.64%     242.540ms        88.51%     269.541ms     269.541ms       0.000us         0.00%      64.697ms      64.697ms             1  
         

In [55]:
print(p2.key_averages().table(sort_by="cuda_time_total"))


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference         0.00%       0.000us         0.00%       0.000us       0.000us      44.951ms        75.29%      44.951ms      44.951ms             1  
                                        model_inference        85.03%     228.484ms       100.00%     268.706ms     268.706ms       0.000us         0.00%      14.754ms      14.754ms             1  
         

In [56]:
p1, p2 = gen_and_test_gpu(num_field = 34,prefix = 29, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 34, prefix: 29, batch :4096


STAGE:2025-02-07 19:54:17 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:54:18 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:54:18 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
STAGE:2025-02-07 19:54:18 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


258.59737396240234


STAGE:2025-02-07 19:54:18 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:54:18 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


279.2832851409912


In [57]:
print(p1.key_averages().table(sort_by="cuda_time_total"))


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference         0.00%       0.000us         0.00%       0.000us       0.000us      35.174ms        51.34%      35.174ms      35.174ms             1  
                                        model_inference        88.30%     231.538ms        98.69%     258.778ms     258.778ms       0.000us         0.00%      33.334ms      33.334ms             1  
         

In [58]:
print(p2.key_averages().table(sort_by="cuda_time_total"))


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference         0.00%       0.000us         0.00%       0.000us       0.000us      45.573ms        86.03%      45.573ms      45.573ms             1  
                                        model_inference        85.18%     238.050ms       100.00%     279.455ms     279.455ms       0.000us         0.00%       7.401ms       7.401ms             1  
         

In [59]:
p1, p2 = gen_and_test_gpu(num_field = 22,prefix = 10, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 22, prefix: 10, batch :4096


STAGE:2025-02-07 19:54:19 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:54:19 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:54:19 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


255.65147399902344


STAGE:2025-02-07 19:54:20 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:54:20 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:54:20 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


274.85132217407227


In [60]:
print(p1.key_averages().table(sort_by="cuda_time_total"))


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference         0.00%       0.000us         0.00%       0.000us       0.000us      32.045ms        58.63%      32.045ms      32.045ms             1  
                                        model_inference        88.97%     227.638ms       100.00%     255.842ms     255.842ms       0.000us         0.00%      22.608ms      22.608ms             1  
         

In [61]:
print(p2.key_averages().table(sort_by="cuda_time_total"))


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference         0.00%       0.000us         0.00%       0.000us       0.000us      46.072ms        75.51%      46.072ms      46.072ms             1  
                                        model_inference        84.80%     233.237ms       100.00%     275.053ms     275.053ms       0.000us         0.00%      14.942ms      14.942ms             1  
         

In [62]:
p1, p2 = gen_and_test_gpu(num_field = 22,prefix = 10, batch = 8192, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 22, prefix: 10, batch :8192


STAGE:2025-02-07 19:54:21 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:54:21 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:54:21 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


249.2961883544922


STAGE:2025-02-07 19:54:22 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:54:22 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:54:22 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


272.2022533416748


In [63]:
print(p1.key_averages().table(sort_by="cuda_time_total"))


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference         0.00%       0.000us         0.00%       0.000us       0.000us      44.995ms        51.03%      44.995ms      44.995ms             1  
                                        model_inference        84.30%     222.510ms        94.50%     249.433ms     249.433ms       0.000us         0.00%      43.181ms      43.181ms             1  
         

In [64]:
print(p2.key_averages().table(sort_by="cuda_time_total"))


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference         0.00%       0.000us         0.00%       0.000us       0.000us      46.249ms        62.93%      46.249ms      46.249ms             1  
                                        model_inference        84.55%     230.346ms       100.00%     272.414ms     272.414ms       0.000us         0.00%      27.240ms      27.240ms             1  
         

In [65]:
ori, modify = workload_FM(22,10,32)

In [66]:
traced_model = torch.jit.trace(ori, torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))


In [67]:
compiled_model = torch.compile(traced_model, backend="inductor")

In [68]:
compiled_model.eval()
total_time = 0
t = torch.randint(low=0, high=88, size=(4096 ,22), dtype=torch.long)

with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = compiled_model(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time += elapsed_time
total_time * 1000

STAGE:2025-02-07 19:54:23 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:54:23 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:54:23 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


122.13635444641113

In [69]:
print(prof.key_averages().table(sort_by="cpu_time_total"))


-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
        model_inference         6.97%       8.553ms       100.00%     122.791ms     122.791ms             1  
                forward         3.43%       4.214ms        93.03%     114.238ms       1.142ms           100  
        aten::embedding         1.65%       2.031ms        30.41%      37.342ms     186.710us           200  
     aten::index_select        27.16%      33.348ms        28.40%      34.868ms     174.340us           200  
              aten::sum        16.97%      20.839ms        22.08%      27.117ms      67.793us           400  
              aten::add        15.90%      19.521ms        15.90%      19.521ms      48.803us           400  
          

In [70]:
traced_model = torch.jit.trace(modify, torch.randint(low=0, high=88, size=(4096 ,22), dtype=torch.long))


In [71]:
compiled_model = torch.compile(traced_model, backend="inductor")

In [72]:
compiled_model.eval()
total_time = 0
t = torch.randint(low=0, high=88, size=(4096 ,22), dtype=torch.long)

with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = compiled_model(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time += elapsed_time
total_time * 1000

STAGE:2025-02-07 19:54:24 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:54:24 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:54:24 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


128.45373153686523

In [73]:
print(prof.key_averages().table(sort_by="cpu_time_total"))


-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
        model_inference         7.23%       9.322ms       100.00%     128.853ms     128.853ms             1  
                forward         4.75%       6.121ms        92.76%     119.518ms       1.195ms           100  
        aten::embedding         1.70%       2.186ms        32.26%      41.574ms     138.580us           300  
     aten::index_select        22.55%      29.058ms        23.96%      30.875ms     102.917us           300  
              aten::add        22.92%      29.529ms        22.92%      29.529ms      49.215us           600  
              aten::sum        16.03%      20.650ms        20.60%      26.542ms      44.237us           600  
          

In [74]:
ori, modify = workload_FM(22,10,64)

In [75]:
traced_model = torch.jit.trace(ori, torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
compiled_model = torch.compile(traced_model, backend="inductor")

In [76]:
compiled_model.eval()
total_time = 0
t = torch.randint(low=0, high=88, size=(4096 ,22), dtype=torch.long)

with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = compiled_model(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time += elapsed_time
total_time * 1000

STAGE:2025-02-07 19:54:25 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:54:25 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:54:25 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


140.59686660766602

In [77]:
print(prof.key_averages().table(sort_by="cpu_time_total"))


-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
        model_inference         6.14%       8.646ms       100.00%     140.914ms     140.914ms             1  
                forward         3.02%       4.258ms        93.86%     132.260ms       1.323ms           100  
        aten::embedding         1.64%       2.311ms        29.42%      41.451ms     207.255us           200  
     aten::index_select        26.25%      36.993ms        27.41%      38.629ms     193.145us           200  
              aten::sum        22.39%      31.548ms        25.69%      36.203ms      90.507us           400  
              aten::pow        17.64%      24.861ms        17.66%      24.881ms     124.405us           200  
          

In [78]:
traced_model = torch.jit.trace(modify, torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
compiled_model = torch.compile(traced_model, backend="inductor")

In [79]:
compiled_model.eval()
total_time = 0
t = torch.randint(low=0, high=88, size=(4096 ,22), dtype=torch.long)

with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = compiled_model(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time += elapsed_time
total_time * 1000

STAGE:2025-02-07 19:54:25 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:54:26 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:54:26 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


139.2350196838379

In [80]:
print(prof.key_averages().table(sort_by="cpu_time_total"))


-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
        model_inference         6.92%       9.657ms       100.00%     139.538ms     139.538ms             1  
                forward         4.37%       6.095ms        93.08%     129.881ms       1.299ms           100  
        aten::embedding         1.67%       2.333ms        31.24%      43.591ms     145.303us           300  
     aten::index_select        22.35%      31.180ms        23.80%      33.217ms     110.723us           300  
              aten::sum        18.74%      26.143ms        23.31%      32.523ms      54.205us           600  
              aten::add        19.27%      26.888ms        19.27%      26.888ms      44.813us           600  
          

In [81]:
ori, modify = workload_FM(34,29,32)

In [82]:
traced_model = torch.jit.trace(ori, torch.randint(low=0, high=88, size=(4096,34), dtype=torch.long))
compiled_model = torch.compile(traced_model, backend="inductor")

In [83]:
compiled_model.eval()
total_time = 0
t = torch.randint(low=0, high=88, size=(4096 ,34), dtype=torch.long)

with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = compiled_model(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time += elapsed_time
total_time * 1000

STAGE:2025-02-07 19:54:27 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:54:27 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:54:27 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


125.51569938659668

In [84]:
print(prof.key_averages().table(sort_by="cpu_time_total"))


-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
        model_inference         6.91%       8.683ms       100.00%     125.728ms     125.728ms             1  
                forward         3.49%       4.385ms        93.09%     117.045ms       1.170ms           100  
        aten::embedding         1.85%       2.331ms        30.07%      37.812ms     189.060us           200  
     aten::index_select        26.50%      33.320ms        27.83%      34.987ms     174.935us           200  
              aten::sum        19.52%      24.540ms        24.67%      31.018ms      77.545us           400  
              aten::pow        13.92%      17.502ms        13.93%      17.513ms      87.565us           200  
          

In [85]:
traced_model = torch.jit.trace(modify, torch.randint(low=0, high=88, size=(4096,34), dtype=torch.long))
compiled_model = torch.compile(traced_model, backend="inductor")

In [86]:
compiled_model.eval()
total_time = 0
t = torch.randint(low=0, high=88, size=(4096 ,34), dtype=torch.long)

with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = compiled_model(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time += elapsed_time
total_time * 1000

STAGE:2025-02-07 19:54:27 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:54:27 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:54:27 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


137.09211349487305

In [87]:
print(prof.key_averages().table(sort_by="cpu_time_total"))


-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
        model_inference         6.74%       9.251ms       100.00%     137.322ms     137.322ms             1  
                forward         4.62%       6.344ms        93.26%     128.071ms       1.281ms           100  
        aten::embedding         1.59%       2.185ms        33.42%      45.897ms     152.990us           300  
     aten::index_select        20.02%      27.487ms        21.33%      29.296ms      97.653us           300  
              aten::add        21.29%      29.235ms        21.29%      29.235ms      48.725us           600  
              aten::sum        16.61%      22.813ms        20.78%      28.536ms      47.560us           600  
          

In [88]:
ori, modify = workload_FM(34,29,64)

In [89]:
traced_model = torch.jit.trace(ori, torch.randint(low=0, high=88, size=(4096,34), dtype=torch.long))
compiled_model = torch.compile(traced_model, backend="inductor")

In [90]:
compiled_model.eval()
total_time = 0
t = torch.randint(low=0, high=88, size=(4096 ,34), dtype=torch.long)

with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = compiled_model(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time += elapsed_time
total_time * 1000
print(prof.key_averages().table(sort_by="cpu_time_total"))


STAGE:2025-02-07 19:54:28 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:54:29 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:54:29 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
        model_inference         6.18%      10.310ms       100.00%     166.759ms     166.759ms             1  
                forward         3.10%       5.172ms        93.82%     156.449ms       1.564ms           100  
              aten::sum        23.25%      38.768ms        26.66%      44.466ms     111.165us           400  
        aten::embedding         1.53%       2.546ms        26.14%      43.592ms     217.960us           200  
     aten::index_select        23.21%      38.700ms        24.30%      40.530ms     202.650us           200  
              aten::pow        22.68%      37.822ms        22.69%      37.833ms     189.165us           200  
          

In [91]:
traced_model = torch.jit.trace(modify, torch.randint(low=0, high=88, size=(4096,34), dtype=torch.long))
compiled_model = torch.compile(traced_model, backend="inductor")

In [92]:
compiled_model.eval()
total_time = 0
t = torch.randint(low=0, high=88, size=(4096 ,34), dtype=torch.long)

with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = compiled_model(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time += elapsed_time
total_time * 1000
print(prof.key_averages().table(sort_by="cpu_time_total"))

STAGE:2025-02-07 19:54:29 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:54:29 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:54:29 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
        model_inference         6.33%       9.100ms       100.00%     143.811ms     143.811ms             1  
                forward         4.33%       6.222ms        93.67%     134.702ms       1.347ms           100  
        aten::embedding         1.53%       2.197ms        30.96%      44.522ms     148.407us           300  
              aten::sum        20.88%      30.028ms        24.83%      35.704ms      59.507us           600  
              aten::add        19.66%      28.267ms        19.66%      28.267ms      47.112us           600  
     aten::index_select        17.16%      24.673ms        18.42%      26.495ms      88.317us           300  
          

In [93]:
ori, modify = workload_FM(100,40,32)

In [94]:
traced_model = torch.jit.trace(ori, torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
compiled_model = torch.compile(traced_model, backend="inductor")

In [95]:
compiled_model.eval()
total_time = 0
for i in range(100):
    t = torch.randint(low=0, high=88, size=(4096 ,100), dtype=torch.long)
    start_time = time.time()  # 开始计时
    with torch.no_grad():
      soutput = compiled_model(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time += elapsed_time
total_time * 1000

290.47369956970215

In [96]:
compiled_model.eval()
total_time = 0
t = torch.randint(low=0, high=88, size=(4096 ,100), dtype=torch.long)

with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = compiled_model(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time += elapsed_time
total_time * 1000

STAGE:2025-02-07 19:54:31 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:54:32 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:54:32 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


281.5871238708496

In [97]:
print(prof.key_averages().table(sort_by="cpu_time_total"))


-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
        model_inference         6.02%      16.995ms       100.00%     282.179ms     282.179ms             1  
                forward         2.89%       8.169ms        93.98%     265.184ms       2.652ms           100  
        aten::embedding         1.20%       3.394ms        26.43%      74.578ms     372.890us           200  
              aten::sum        22.84%      64.442ms        25.70%      72.524ms     181.310us           400  
              aten::pow        25.18%      71.047ms        25.21%      71.135ms     355.675us           200  
     aten::index_select        24.11%      68.027ms        24.94%      70.365ms     351.825us           200  
          

In [98]:
traced_model = torch.jit.trace(modify, torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
compiled_model = torch.compile(traced_model, backend="inductor")

In [99]:
compiled_model.eval()
total_time = 0
for i in range(100):
    t = torch.randint(low=0, high=88, size=(4096 ,100), dtype=torch.long)
    start_time = time.time()  # 开始计时
    with torch.no_grad():
      soutput = compiled_model(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time += elapsed_time
total_time * 1000

188.47107887268066

In [100]:
compiled_model.eval()
total_time = 0
t = torch.randint(low=0, high=88, size=(4096 ,100), dtype=torch.long)

with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = compiled_model(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time += elapsed_time
total_time * 1000

STAGE:2025-02-07 19:54:33 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-07 19:54:33 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-07 19:54:33 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


174.88574981689453

In [101]:
print(prof.key_averages().table(sort_by="cpu_time_total"))


-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
        model_inference         4.04%       7.071ms       100.00%     175.148ms     175.148ms             1  
                forward         4.12%       7.224ms        95.96%     168.077ms       1.681ms           100  
        aten::embedding         1.34%       2.344ms        33.99%      59.541ms     198.470us           300  
     aten::index_select        24.44%      42.799ms        25.45%      44.576ms     148.587us           300  
              aten::sum        21.22%      37.171ms        24.63%      43.147ms      71.912us           600  
              aten::add        15.13%      26.500ms        15.13%      26.500ms      44.167us           600  
          

In [102]:
traced_model = torch.jit.trace(modify, torch.randint(low=0, high=88, size=(4096,34), dtype=torch.long))
compiled_model = torch.compile(traced_model, backend="inductor")

Traceback (most recent call last):
  File "/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph_module.py", line 304, in __call__
    return super(self.cls, obj).__call__(*args, **kwargs)  # type: ignore[misc]
  File "/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1522, in _slow_forward
    result = self.forward(*input, **kwargs)
  File "<eval_with_key>.257 from /home/yssun/pytorch-fm/torchfm/model/test_fx/exp/../../fm.py:23 in forward", line 6, in forward
    add = x + linear_offsets;  linear_offsets = None
RuntimeError: The size of tensor a (34) 

RuntimeError: The size of tensor a (34) must match the size of tensor b (100) at non-singleton dimension 1

In [192]:
compiled_model.eval()
total_time = 0
for i in range(100):
    t = torch.randint(low=0, high=88, size=(4096 ,34), dtype=torch.long)
    start_time = time.time()  # 开始计时
    with torch.no_grad():
      soutput = compiled_model(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time += elapsed_time
total_time * 1000

130.51867485046387

In [206]:
traced_model.eval()
total_time = 0
for i in range(100):
    t = torch.randint(low=0, high=88, size=(4096 ,34), dtype=torch.long)
    start_time = time.time()  # 开始计时
    with torch.no_grad():
      soutput = traced_model(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time += elapsed_time
total_time * 1000

124.3293285369873

In [89]:
ori, modify = workload_FM(22,10,32)

now gen workload of wdl with config: dim: 32, num_field: 22, prefix: 10


In [96]:
traced_model = torch.jit.trace(ori, torch.randint(low=0, high=88, size=(2048,22), dtype=torch.long))
total_time = 0
for i in range(100):
    t = torch.randint(low=0, high=88, size=(2048 ,22), dtype=torch.long)
    start_time = time.time()  # 开始计时
    soutput = traced_model(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time += elapsed_time
total_time * 1000

96.64225578308105

In [97]:
traced_model = torch.jit.trace(modify, torch.randint(low=0, high=88, size=(2048,22), dtype=torch.long))
total_time = 0
for i in range(100):
    t = torch.randint(low=0, high=88, size=(2048 ,22), dtype=torch.long)
    start_time = time.time()  # 开始计时
    soutput = traced_model(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time += elapsed_time
total_time * 1000

102.48804092407227

In [98]:
ori, modify = workload_FM(22,10,64)

now gen workload of wdl with config: dim: 64, num_field: 22, prefix: 10


In [100]:
traced_model = torch.jit.trace(ori, torch.randint(low=0, high=88, size=(2048,22), dtype=torch.long))
total_time = 0
for i in range(100):
    t = torch.randint(low=0, high=88, size=(2048 ,22), dtype=torch.long)
    start_time = time.time()  # 开始计时
    soutput = traced_model(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time += elapsed_time
total_time * 1000

111.09638214111328

In [101]:
traced_model = torch.jit.trace(modify, torch.randint(low=0, high=88, size=(2048,22), dtype=torch.long))
total_time = 0
for i in range(100):
    t = torch.randint(low=0, high=88, size=(2048 ,22), dtype=torch.long)
    start_time = time.time()  # 开始计时
    soutput = traced_model(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time += elapsed_time
total_time * 1000

99.17354583740234

In [108]:
traced_model = torch.jit.trace(ori, torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
total_time = 0
for i in range(100):
    t = torch.randint(low=0, high=88, size=(4096 ,22), dtype=torch.long)
    start_time = time.time()  # 开始计时
    soutput = traced_model(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time += elapsed_time
total_time * 1000

298.5067367553711

In [110]:
traced_model = torch.jit.trace(modify, torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
total_time = 0
for i in range(100):
    t = torch.randint(low=0, high=88, size=(4096 ,22), dtype=torch.long)
    start_time = time.time()  # 开始计时
    soutput = traced_model(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time += elapsed_time
total_time * 1000

116.41645431518555

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [52]:
ori.to(device)

FactorizationMachineModel(
  (linear): Module(
    (fc): Embedding(2200, 1)
  )
  (embedding): Module(
    (embedding): Embedding(2200, 32)
  )
)

In [53]:
traced_model = torch.jit.trace(ori, torch.randint(low=0, high=88, size=(4096 * 2,22), dtype=torch.long).to(device))


In [58]:
%timeit -n 1 -r 30  output = traced_model(torch.randint(low=0, high=88, size=(4096 * 2,22), dtype=torch.long).to(device))


3.06 ms ± 982 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [59]:
modify.to(device)

FactorizationMachineModel(
  (linear): Module(
    (fc): Embedding(2200, 1)
  )
  (embedding): Module()
  (embed): Embedding(2200, 32)
)

In [60]:
traced_model = torch.jit.trace(modify, torch.randint(low=0, high=88, size=(4096 * 2,22), dtype=torch.long).to(device))


In [66]:
%timeit -n 1 -r 30  output = traced_model(torch.randint(low=0, high=88, size=(4096 * 2,22), dtype=torch.long).to(device))


3.06 ms ± 880 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [67]:
ori, modify = workload_FM(22,10,64)

now gen workload of wdl with config: dim: 64, num_field: 22, prefix: 10


In [68]:
ori.to(device)

FactorizationMachineModel(
  (linear): Module(
    (fc): Embedding(2200, 1)
  )
  (embedding): Module(
    (embedding): Embedding(2200, 64)
  )
)

In [107]:
traced_model = torch.jit.trace(ori, torch.randint(low=0, high=88, size=(4096 * 2,22), dtype=torch.long).to(device))


In [108]:
t = torch.randint(low=0, high=88, size=(4096 * 2,22), dtype=torch.long).to(device)

In [111]:
t = torch.randint(low=0, high=88, size=(4096 * 2,22), dtype=torch.long).to(device)
%timeit -n 1 -r 30  output = traced_model(t)


311 µs ± 142 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [116]:
total_time = 0
for i in range(30):
    t = torch.randint(low=0, high=88, size=(4096 * 2,22), dtype=torch.long).to(device)
    start_time = time.time()  # 开始计时
    soutput = traced_model(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time += elapsed_time
total_time * 1000

10.293006896972656

In [74]:
modify.to(device)

FactorizationMachineModel(
  (linear): Module(
    (fc): Embedding(2200, 1)
  )
  (embedding): Module()
  (embed): Embedding(2200, 64)
)

In [129]:
traced_model = torch.jit.trace(modify, torch.randint(low=0, high=88, size=(4096 * 2,22), dtype=torch.long).to(device))


In [130]:
t = torch.randint(low=0, high=88, size=(4096 * 2,22), dtype=torch.long).to(device)

In [134]:
total_time = 0
for i in range(30):
    t = torch.randint(low=0, high=88, size=(4096 * 2,22), dtype=torch.long).to(device)
    start_time = time.time()  # 开始计时
    soutput = traced_model(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time += elapsed_time
total_time * 1000

11.205434799194336

In [104]:
%timeit -n 1 -r 30  output = traced_model(t)


The slowest run took 4.58 times longer than the fastest. This could mean that an intermediate result is being cached.
418 µs ± 165 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [6]:
ori, modify = workload_FM(110,50,64)

now gen workload of wdl with config: dim: 64, num_field: 110, prefix: 50


In [19]:
ori.to(device)

FactorizationMachineModel(
  (linear): Module(
    (fc): Embedding(11000, 1)
  )
  (embedding): Module(
    (embedding): Embedding(11000, 64)
  )
)

In [20]:
traced_model = torch.jit.trace(ori, torch.randint(low=0, high=88, size=(4096 * 2 ,110), dtype=torch.long).to(device))


In [22]:
total_time = 0
for i in range(30):
    t = torch.randint(low=0, high=88, size=(4096 * 2,110), dtype=torch.long).to(device)
    start_time = time.time()  # 开始计时
    soutput = traced_model(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time += elapsed_time
total_time * 1000

7.686614990234375

In [23]:
modify.to(device)

FactorizationMachineModel(
  (linear): Module(
    (fc): Embedding(11000, 1)
  )
  (embedding): Module()
  (embed): Embedding(11000, 64)
)

In [24]:
traced_model = torch.jit.trace(modify, torch.randint(low=0, high=88, size=(4096 * 2 ,110), dtype=torch.long).to(device))


In [26]:
total_time = 0
for i in range(30):
    t = torch.randint(low=0, high=88, size=(4096 * 2 ,110), dtype=torch.long).to(device)
    start_time = time.time()  # 开始计时
    soutput = traced_model(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time += elapsed_time
total_time * 1000

10.19430160522461

In [149]:
ori, modify = workload_FM(34,29,128)

now gen workload of wdl with config: dim: 128, num_field: 34, prefix: 29


In [150]:
ori.to(device)

FactorizationMachineModel(
  (linear): Module(
    (fc): Embedding(3400, 1)
  )
  (embedding): Module(
    (embedding): Embedding(3400, 128)
  )
)

In [151]:
traced_model = torch.jit.trace(ori, torch.randint(low=0, high=88, size=(4096 * 2,34), dtype=torch.long).to(device))


In [153]:
total_time = 0
for i in range(30):
    t = torch.randint(low=0, high=88, size=(4096 * 2,34), dtype=torch.long).to(device)
    start_time = time.time()  # 开始计时
    soutput = traced_model(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time += elapsed_time
total_time * 1000

10.440826416015625

In [154]:
modify.to(device)

FactorizationMachineModel(
  (linear): Module(
    (fc): Embedding(3400, 1)
  )
  (embedding): Module()
  (embed): Embedding(3400, 128)
)

In [155]:
traced_model = torch.jit.trace(modify, torch.randint(low=0, high=88, size=(4096 * 2,34), dtype=torch.long).to(device))


In [159]:
total_time = 0
for i in range(30):
    t = torch.randint(low=0, high=88, size=(4096 * 2,34), dtype=torch.long).to(device)
    start_time = time.time()  # 开始计时
    soutput = traced_model(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time += elapsed_time
total_time * 1000

12.769222259521484

In [149]:
p1, p2 = gen_and_test_KFLOPS(num_field = 100,prefix = 10, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 10, batch :4096


STAGE:2025-02-08 01:19:05 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 01:19:07 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 01:19:07 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(17.118701934814453, 0.3701509728216479)


STAGE:2025-02-08 01:19:07 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 01:19:08 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 01:19:08 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(3.230431079864502, 1.686831036630565)


In [150]:
print(p1.key_averages().table(sort_by="self_cpu_time_total"))

-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  Total KFLOPs  
-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
              aten::pow        46.47%     795.709ms        46.48%     795.822ms       3.979ms           200            --  
              aten::sum        29.13%     498.883ms        30.10%     515.376ms       1.288ms           400            --  
     aten::index_select        16.62%     284.552ms        16.75%     286.831ms       1.434ms           200            --  
              aten::add         3.27%      55.910ms         3.27%      55.910ms     139.775us           400     82739.200  
        model_inference         1.09%      18.647ms       100.00%        1.712s        1.712s             1            --  
        

In [151]:
print(p2.key_averages().table(sort_by="self_cpu_time_total"))

-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  Total KFLOPs  
-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
              aten::pow        23.86%      77.179ms        23.87%      77.212ms     257.373us           300            --  
              aten::add        19.37%      62.661ms        19.37%      62.661ms     104.435us           600     83558.400  
     aten::index_select        19.32%      62.504ms        19.92%      64.446ms     214.820us           300            --  
              aten::sum        15.96%      51.639ms        18.26%      59.080ms      98.467us           600            --  
        model_inference         5.20%      16.836ms       100.00%     323.498ms     323.498ms             1            --  
        

In [153]:
p1, p2 = gen_and_test_KFLOPS(num_field = 100,prefix = 5, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 5, batch :4096


STAGE:2025-02-08 01:20:28 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 01:20:30 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 01:20:30 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(16.083087921142578, 0.2523498727896367)


STAGE:2025-02-08 01:20:30 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 01:20:32 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 01:20:32 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(15.818719863891602, 0.27503584246915125)


In [ ]:
p1, p2 = gen_and_test_KFLOPS(num_field = 100,prefix = 5, batch = 4096, dim = 32)

In [162]:
p1, p2 = gen_and_test_KFLOPS(num_field = 100,prefix = 5, batch = 2048, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 5, batch :2048


STAGE:2025-02-08 01:29:24 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 01:29:24 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 01:29:24 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.384519577026367, 0.1783459845228208)


STAGE:2025-02-08 01:29:24 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 01:29:24 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 01:29:24 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.3348379135131836, 0.20727670741962356)


In [163]:
p1, p2 = gen_and_test_KFLOPS(num_field = 100,prefix = 10, batch = 2048, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 10, batch :2048


STAGE:2025-02-08 01:29:26 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


STAGE:2025-02-08 01:29:27 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 01:29:27 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.950448989868164, 1.058004542846902)


STAGE:2025-02-08 01:29:27 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 01:29:28 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 01:29:28 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.9997572898864746, 0.2690364102534204)


In [161]:
p1, p2 = gen_and_test_KFLOPS(num_field = 100,prefix = 30, batch = 2048, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 30, batch :2048


STAGE:2025-02-08 01:29:19 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 01:29:20 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 01:29:20 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.27262020111084, 0.10422725360967888)


STAGE:2025-02-08 01:29:20 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 01:29:20 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 01:29:20 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.8941926956176758, 0.284242965312842)


In [164]:
p1, p2 = gen_and_test_KFLOPS(num_field = 100,prefix = 80, batch = 2048, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 80, batch :2048


STAGE:2025-02-08 01:29:38 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 01:29:38 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 01:29:38 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.1103262901306152, 0.08136321227425469)


STAGE:2025-02-08 01:29:38 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 01:29:39 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 01:29:39 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.4752578735351562, 0.09373878342557873)


In [167]:
p1, p2 = gen_and_test_KFLOPS(num_field = 100,prefix = 10, batch = 2048, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 100, prefix: 10, batch :2048


STAGE:2025-02-08 01:30:04 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 01:30:06 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 01:30:06 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(14.344596862792969, 0.10257133908453397)


STAGE:2025-02-08 01:30:06 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 01:30:08 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 01:30:08 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(13.439013957977295, 1.2807581045933603)


In [166]:
p1, p2 = gen_and_test_KFLOPS(num_field = 100,prefix = 80, batch = 2048, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 100, prefix: 80, batch :2048


STAGE:2025-02-08 01:29:58 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 01:29:59 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 01:29:59 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(14.297704696655273, 0.13789806721433706)


STAGE:2025-02-08 01:30:00 2592637:2592637 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 01:30:00 2592637:2592637 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 01:30:00 2592637:2592637 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.6067957878112793, 0.2092130067637754)
